In [1]:
%%configure -f
{"defaultLakehouse": {"name": "lh_cfg"}}

# Set variables

In [2]:
is_debug       = False
medallion_name = "Power BI"

if is_debug:
    print(f"is_debug:       {is_debug}")
    print(f"medallion_name: {medallion_name}")

# Imports

In [3]:
import delta.tables                       as dlt
import json                               as j
from   types       import SimpleNamespace as sn
from   pyspark.sql import functions       as sf
from   pyspark.sql import DataFrame       as df
from   pyspark.sql import types           as st
from   pyspark.sql import Row
from   inspect     import stack           as stk
import time                               as t
import datetime                           as dt
import pytz                               as pt
import re
import unicodedata                        as ucd
import pandas                             as pd
import sys
import requests                           as req
if is_debug:
  from pyspark.sql import SparkSession    as ss

# Get parameters from "pl_master"

In [4]:
global_parameter = "global_parameter from pl_master"
workspace_id     = "workspace_id from pl_master"

In [5]:
# Current session info
if is_debug: display(ss.getActiveSession())

In [6]:
# Freeze this
# Simulate parameters from "pl_master".

workspace_id = "abc"

global_parameter = """
{
    "days_to_keep_log": 365
    , "send_success_email": "True"
    , "send_error_email": "True"
    , "email_on_success": "example@example.com"
    , "email_on_error": "example@example.com"
    , "time_zone_nb": "Canada/Eastern"
    , "time_zone_pl": "Eastern Standard Time"
    , "mssql_isolation_level": "READ_UNCOMMITTED"
    , "projet_documentation_data_file_name": "configuration"
    , "sql_server": "sql-server.database.windows.net"
    , "key_vault_url": "https://kv-keyvault.vault.azure.net/"
    , "kv_test_key": "kv-my-key"
    , "bearer_token_url": "https://api.fabric.microsoft.com/"
    , "abfs_path_lh_cfg": "abfss://abc@onelake.dfs.fabric.microsoft.com/abc"
    , "abfs_path_lh_log": "abfss://abc@onelake.dfs.fabric.microsoft.com/abc"
    , "abfs_path_lh_silver": "abfss://abc@onelake.dfs.fabric.microsoft.com/1abc"
    , "abfs_path_lh_bronze": "abfss://abc@onelake.dfs.fabric.microsoft.com/abc"
    , "abfs_path_lh_AdventureWorks": "abfss://abc@onelake.dfs.fabric.microsoft.com/abc"
	, "my_dataverse": "abfss://abc@onelake.dfs.fabric.microsoft.com/abc"
    , "abfs_path_lh_gold":"abfss://abc@onelake.dfs.fabric.microsoft.com/abc"
	, "now_datetime": "2024-05-07 14:09:09.367"
    , "process_timestamp": "20240507_140909367"
}
"""

# Set parameters from "pl_master" in variables

In [7]:
# Create local variables from the parameters, paseed by "pl_master" into namespace "global_parameter"

global_parameter = j.loads(global_parameter) # Convert string to mapping
global_parameter = sn(**global_parameter)    # Create variables from mapping

if is_debug:
    print(type(global_parameter))
    print(f"global_parameter.days_to_keep_log:                    {global_parameter.days_to_keep_log}")
    print(f"global_parameter.send_success_email:                  {global_parameter.send_success_email}")
    print(f"global_parameter.send_error_email:                    {global_parameter.send_error_email}")
    print(f"global_parameter.email_on_success:                    {global_parameter.email_on_success}")
    print(f"global_parameter.email_on_error:                      {global_parameter.email_on_error}")
    print(f"global_parameter.time_zone_nb:                        {global_parameter.time_zone_nb}")
    print(f"global_parameter.time_zone_pl:                        {global_parameter.time_zone_pl}")
    print(f"global_parameter.mssql_isolation_level:               {global_parameter.mssql_isolation_level}")
    print(f"global_parameter.projet_documentation_data_file_name: {global_parameter.projet_documentation_data_file_name}")
    print(f"sql_server:                                           {global_parameter.sql_server}")
    print(f"key_vault_url:                                        {global_parameter.key_vault_url}")
    print(f"kv_test_key:                                          {global_parameter.kv_test_key}")
    print(f"bearer_token_url:                                     {global_parameter.bearer_token_url}")
    print(f"global_parameter.abfs_path_lh_cfg:                    {global_parameter.abfs_path_lh_cfg}")
    print(f"global_parameter.abfs_path_lh_log:                    {global_parameter.abfs_path_lh_log}")
    print(f"global_parameter.abfs_path_lh_silver:                 {global_parameter.abfs_path_lh_silver}")    
    print(f"global_parameter.abfs_path_lh_bronze:                 {global_parameter.abfs_path_lh_bronze}")
    print(f"abfs_path_lh_AdventureWorks:                          {global_parameter.abfs_path_lh_AdventureWorks}")    
    print(f"global_parameter.my_dataverse:                        {global_parameter.my_dataverse}")
    print(f"abfs_path_lh_gold:                                    {global_parameter.abfs_path_lh_gold}")
    print(f"global_parameter.now_datetime:                        {global_parameter.now_datetime}")
    print(f"global_parameter.process_timestamp:                   {global_parameter.process_timestamp}")

In [ ]:
# inject environment in global parameter

workspace_name = notebookutils.runtime.context.get("currentWorkspaceName")
if is_debug: print(f"workspace_name: {workspace_name}")

# Presumption:
workspace_name = "zPL Concept - Fabric Framework (DEV)"  # DEV
#workspace_name = "zPL Concept - Fabric Framework (TEST)" # TEST
#workspace_name = "zPL Concept - Fabric Framework"        # PROD

# Set variable "environment"
env = workspace_name[-6:]
if env.find("(") == -1: global_parameter.environment = "PROD"
else:                   global_parameter.environment = env[(env.index("(") + 1):env.index(")")]

if is_debug:
    print("global_parameter:")
    display(global_parameter)

# Function

## Run notebooks "system_cfg" and "system functions"

In [9]:
#%run nb_system_cfg

In [10]:
%run nb_system_function

In [ ]:
%run nb_power_bi_function

In [ ]:
# Freeze this
# Reset now_datetime, process_timestamp

global_parameter.now_datetime      = fn_get_now("datetime")[1]
global_parameter.process_timestamp = fn_get_now("string")[1]

if is_debug:
    print(f"global_parameter.now_datetime:      {global_parameter.now_datetime}")
    print(f"global_parameter.process_timestamp: {global_parameter.process_timestamp}")

# Operation

## Get the frequencies that need to be refreshed now

In [ ]:
#now_datetime = fn_get_now("datetime")[1]
#now_datetime = global_parameter.now_datetime
now_datetime = "2024-04-01 01:11:27.020" # kill me. Simulate first day on month @ 1 AM.

sql_code = f"""SELECT DISTINCT Z.`frequency`
FROM
  (
    SELECT
      *
      , CASE WHEN  Y.`frequency` = 'Hourly'                                                                                                                                      THEN 1 END AS H
      , CASE WHEN (Y.`frequency` = 'Daily'     AND HOUR('{now_datetime}') = 1                                                                                                  ) THEN 1 END AS D
      , CASE WHEN (Y.`frequency` = 'Weekly'    AND HOUR('{now_datetime}') = 1 AND DATE_FORMAT('{now_datetime}', 'EEEE') = 'Monday'                                             ) THEN 1 END AS W
      , CASE WHEN (Y.`frequency` = 'Monthly'   AND DAY('{now_datetime}' ) = 1 AND HOUR('{now_datetime}')                = 1                                                    ) THEN 1 END AS M
      , CASE WHEN (Y.`frequency` = 'Quarterly' AND DAY('{now_datetime}' ) = 1 AND HOUR('{now_datetime}')                = 1        AND MONTH('{now_datetime}') IN (1, 4, 7, 10)) THEN 1 END AS Q
      , CASE WHEN (Y.`frequency` = 'Yearly'    AND DAY('{now_datetime}' ) = 1 AND HOUR('{now_datetime}')                = 1        AND MONTH('{now_datetime}') = 1             ) THEN 1 END AS Y
    FROM
      (
              SELECT `frequency` FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_csv`        WHERE `is_extracted` = 1
        UNION SELECT `frequency` FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_excel`      WHERE `is_extracted` = 1
        UNION SELECT `frequency` FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_json`       WHERE `is_extracted` = 1
        UNION SELECT `frequency` FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_lakehouse`  WHERE `is_extracted` = 1
        UNION SELECT `frequency` FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_sql_server` WHERE `is_extracted` = 1
        UNION SELECT `frequency` FROM delta.`{global_parameter.abfs_path_lh_cfg}/Tables/extract_object_api`        WHERE `is_extracted` = 1
      ) AS Y
  ) AS Z
WHERE COALESCE(Z.H, Z.D, Z.W, Z.M, Z.Q, Z.Y) = 1;"""
sdf_frequency = fn_execute_spark_sql(sql_code)[2]

if is_debug:
    print(f"sql_code: {sql_code}")
    display(sdf_frequency)

## Set "list_frequency"

In [ ]:
list_frequency = []
for row in sdf_frequency.select(sdf_frequency["frequency"]).distinct().collect(): list_frequency.append(row.frequency)

# Insert "list_technology" and "list_frequency" in "sdf_log"
locals_                    = {}
locals_["list_frequency"]  = list_frequency
locals_                    = j.dumps(locals_)
fn_local_log_insert(global_parameter.process_timestamp, medallion_name, "nb_power_bi_master", locals_, "Success", "")

if is_debug: print(f"list_frequency: {type(list_frequency)} | {list_frequency}")

In [ ]:
# Power BI Refresh

fn_exec_async_pbi_refresh(list_frequency)

In [ ]:
# Display local log

pdf_log.reset_index(drop = True, inplace = True)
display(pdf_log.sort_values(by = "logged_datetime", ascending = False))